In [1]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from sqlalchemy import create_engine
import pandas as pd
import json
import requests
import io
import calendar

# 1. Transform, Format and Clean Data. 

# 2. Seperate into dimensions and facts

# 3. Save the data frames as CSV  

# 4. Load Data into the Data Warehouse

In [2]:
# Read the JSON config file
config_file_path = 'config.json'
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file) 

# Azure connection string
CONNECTION_STRING = config['AZURE_CONNECTION_STRING']
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)

# Database connection
DATABASE = config['DW_CONNECTION_STRING']

In [3]:
def get_blob_list(container_name):
    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()
    return blob_list

In [4]:
def get_azure_blob_data(container_name, blob):
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob.name)
    stream = blob_client.download_blob()
    blob_content = b""
    for chunk in stream.chunks():
        blob_content += chunk
    return blob_content

In [5]:
def download_file(url):
    response = requests.get(url)
    return io.BytesIO(response.content)

# Retrieving the data from Azure Containers

In [6]:
def get_ppp_loan_data():
    container_name = 'pppdata'
    blob_list = get_blob_list(container_name)
    
    for blob in blob_list:
        if "public_150k_plus" in blob.name:
            print(f"Downloading {blob.name}")
            blob_data = get_azure_blob_data(container_name, blob)
            print(f"Downloaded {blob.name} successfully")
            data = io.BytesIO(blob_data)
            print(f"Reading {blob.name}")
            df_chunks = pd.read_csv(data, chunksize=100000)  # Adjust the chunksize as per your memory capacity
            df_list = []
            for chunk in df_chunks:
                df_list.append(chunk)
            df = pd.concat(df_list)
            return df

In [7]:
def get_naics_data():
    container_name = 'naicsdata'
    blob_list = get_blob_list(container_name)

    for blob in blob_list:
        blob_data = get_azure_blob_data(container_name, blob)
        data = io.BytesIO(blob_data)
        df = pd.read_csv(data)
        return df
    

In [8]:
def get_gdp_data():
    container_name = 'gdpdata'
    blob_list = get_blob_list(container_name)

    for blob in blob_list:
        blob_data = get_azure_blob_data(container_name, blob)
        data = io.BytesIO(blob_data)
        df = pd.read_csv(data)
        return df

# Reformating, and Cleaning the data

In [9]:
def reformat_naics_data():
    df_naics = get_naics_data()
    df_naics.rename(columns={
        'Code': 'NAICS_CODE',
        'Title': 'NAICS_TITLE',
        'Description': 'DESCRIPTION'
    }, inplace=True)
    # Remove all the rows where NAICS_CODE is not a number
    # The NAICS_CODE column has some generic values like "31-33" which are not valid NAICS codes
    df_naics = df_naics[df_naics['NAICS_CODE'].str.isnumeric()]

    df_naics['NAICS_CODE'] = df_naics['NAICS_CODE'].astype(int)
    df_naics['NAICS_TITLE'] = df_naics['NAICS_TITLE'].astype(pd.StringDtype("pyarrow"))
    df_naics['DESCRIPTION'] = df_naics['DESCRIPTION'].astype(pd.StringDtype("pyarrow"))
    
    return df_naics

In [10]:
def reformat_gdp_data():
    df_gdp  = get_gdp_data()
    #Drop all the records where 2017, 2018, 2019, 2020, 2021, 2022 = "(NA)" 
    df_gdp = df_gdp[df_gdp['2017'] != "(NA)"]
    df_gdp = df_gdp[df_gdp['2020'] != "(NA)"]

    # Pivot the data in GDP data
    selected_columns = ['GeoFIPS', 'GeoName', 'Region', 'Description', '2017', '2018', '2019', '2020', '2021', '2022']
    df_gdp = df_gdp[selected_columns]
    pivot_data = df_gdp.melt(id_vars=["GeoFIPS", "GeoName", "Region", "Description"],
                                    value_vars=["2017", "2018", "2019", "2020", "2021", "2022"],
                                    var_name="date_id",
                                    value_name="Value")
    pivot_data = pivot_data.pivot_table(index=["GeoFIPS", "GeoName", "Region", "date_id"], columns="Description", values="Value", aggfunc='first').reset_index()
    pivot_data = pivot_data.sort_values(by=["GeoFIPS", "date_id"])
    pivot_data.rename(columns={
        "Chain-type quantity indexes for real GDP ": "CHAIN_TYPE_INDEX_GDP",
        "Current-dollar GDP (thousands of current dollars) ": "CURRENT_DOLLAR_GDP",
        "Real GDP (thousands of chained 2017 dollars) ": "REAL_GDP",
        "GeoFIPS": "GEOFIPS",
        "GeoName": "GEO_NAME",
        "Description": "Index",
        "date_id": "YEAR_ID"
    }, inplace=True)
    pivot_data['FACTS_GDP_ID'] = range(1, len(pivot_data) + 1)
    final_data = pivot_data.drop(columns='Description', errors='ignore')
    final_data = pivot_data[['FACTS_GDP_ID', 'GEOFIPS', 'GEO_NAME', 'Region', 'YEAR_ID', 'CHAIN_TYPE_INDEX_GDP',
                         'CURRENT_DOLLAR_GDP', 'REAL_GDP']]
    df_gdp = final_data

    # Remove the quation marks from GEOFIPS
    df_gdp['GEOFIPS'] = df_gdp['GEOFIPS'].str.replace('"', '')
    
    # Change the YearID to match the format in the Date Dimension
    df_gdp['YEAR_ID'] = pd.to_datetime(df_gdp['YEAR_ID'], format='%Y').dt.strftime('%Y%m%d%H')
    
    # Change the data types of the columns
    df_gdp['YEAR_ID'] = df_gdp['YEAR_ID'].astype(int)
    df_gdp['GEOFIPS'] = df_gdp['GEOFIPS'].astype(int)
    df_gdp['GEO_NAME'] = df_gdp['GEO_NAME'].astype(pd.StringDtype("pyarrow"))
    df_gdp['Region'] = df_gdp['Region'].astype(pd.StringDtype("pyarrow"))    
    df_gdp['CHAIN_TYPE_INDEX_GDP'] = df_gdp['CHAIN_TYPE_INDEX_GDP'].astype(float)
    df_gdp['CURRENT_DOLLAR_GDP'] = df_gdp['CURRENT_DOLLAR_GDP'].astype(float)
    df_gdp['REAL_GDP'] = df_gdp['REAL_GDP'].astype(float)


    return df_gdp

    
    

In [11]:
def reformat_ppp_loan_data():
    df_ppp = get_ppp_loan_data()


    # Delete the columns that are not required
    df_ppp.drop(columns=[
        'UTILITIES_PROCEED',
        'PAYROLL_PROCEED',
        'MORTGAGE_INTEREST_PROCEED',
        'RENT_PROCEED',
        'REFINANCE_EIDL_PROCEED',
        'HEALTH_CARE_PROCEED',
        'DEBT_INTEREST_PROCEED',
        'RuralUrbanIndicator',
        'HubzoneIndicator',
        'LMIIndicator',
        'ProjectCity',
        'ProjectZip',
        'CD'
    ], inplace=True)
    # Rename the columns to match the SQL table
    df_ppp.rename(columns={
        'LoanNumber': 'LOAN_NUMBER',
        'DateApproved': 'DATE_APPROVED_ID',
        'SBAOfficeCode': 'SBA_OFFICE_CODE',
        'ProcessingMethod': 'PROCESSING_METHOD',
        'BorrowerName': 'BORROWER_NAME',
        'BorrowerAddress': 'BORROWER_ADDRESS',
        'BorrowerCity': 'BORROWER_CITY',
        'BorrowerState': 'BORROWER_STATE',
        'BorrowerZip': 'BORROWER_ZIP',
        'LoanStatusDate': 'LOAN_STATUS_DATE_ID',
        'LoanStatus': 'LOAN_STATUS',
        'Term': 'TERM_MONTH',
        'SBAGuarantyPercentage': 'SBA_GUARANTY_PERCENTAGE',
        'InitialApprovalAmount': 'INITIAL_APPROVAL_AMOUNT',
        'CurrentApprovalAmount': 'CURRENT_APPROVAL_AMOUNT',
        'UndisbursedAmount': 'UNDISBURSED_AMOUNT',
        'FranchiseName': 'FRANCHISE_NAME',
        'ServicingLenderLocationID': 'SERVICING_LENDER_LOCATION_ID',
        'ServicingLenderName': 'SERVICING_LENDER_NAME',
        'ServicingLenderAddress': 'SERVICING_LENDER_ADDRESS',
        'ServicingLenderCity': 'SERVICING_LENDER_CITY',
        'ServicingLenderState': 'SERVICING_LENDER_STATE',
        'ServicingLenderZip': 'SERVICING_LENDER_ZIP',
        'BusinessAgeDescription': 'BUSINESS_AGE_DESCRIPTION',
        'ProjectState': 'PROJECT_STATE',
        'ProjectCountyName': 'PROJECT_COUNTY_NAME',
        'Race': 'RACE',
        'Ethnicity': 'ETHNICITY',
        'Gender': 'GENDER',
        'BusinessType': 'BUSINESS_TYPE',
        'OriginatingLenderLocationID': 'ORIGINATING_LENDER_LOCATION_ID',
        'OriginatingLender': 'ORIGINATING_LENDER',
        'OriginatingLenderCity': 'ORIGINATING_LENDER_CITY',
        'OriginatingLenderState': 'ORIGINATING_LENDER_STATE',
        'Veteran': 'VETERAN',
        'NonProfit': 'NONPROFIT',
        'ForgivenessAmount': 'FORGIVENESS_AMOUNT',
        'ForgivenessDate': 'FORGIVENESS_DATE_ID',
        'JobsReported': 'JOBS_REPORTED',
        'NAICSCode': 'NAICS_CODE'
    }, inplace=True)

    # Droping all the empty rows
    # Drop all the rows where Borrower State is empty
    df_ppp = df_ppp.dropna(subset=['BORROWER_STATE'])

    # Drop all the rows where NAICS_CODE is empty
    df_ppp = df_ppp.dropna(subset=['NAICS_CODE'])

    # Drop all the rows where dates are empty
    df_ppp = df_ppp.dropna(subset=['DATE_APPROVED_ID', 'LOAN_STATUS_DATE_ID', 'FORGIVENESS_DATE_ID'])

    # Drop all the rows where jobs reported is empty
    df_ppp = df_ppp.dropna(subset=['JOBS_REPORTED'])

    # Drop all the rows where business type is empty
    df_ppp = df_ppp.dropna(subset=['BUSINESS_TYPE'])

    # Drop all the rows where business age description is empty
    df_ppp = df_ppp.dropna(subset=['BUSINESS_AGE_DESCRIPTION'])
    # or where the value is Unanswered
    df_ppp = df_ppp[df_ppp['BUSINESS_AGE_DESCRIPTION'] != 'Unanswered']

    

    # Change the Date columns to match the format in the Date Dimension
    df_ppp['FORGIVENESS_DATE_ID'] = pd.to_datetime(df_ppp['FORGIVENESS_DATE_ID']).dt.strftime('%Y%m%d%H')
    df_ppp['DATE_APPROVED_ID'] = pd.to_datetime(df_ppp['DATE_APPROVED_ID']).dt.strftime('%Y%m%d%H')
    df_ppp['LOAN_STATUS_DATE_ID'] = pd.to_datetime(df_ppp['LOAN_STATUS_DATE_ID']).dt.strftime('%Y%m%d%H')
    
    # Change NonProfit to boolean
    df_ppp['NONPROFIT'] = df_ppp['NONPROFIT'].map({'Y': True})
    df_ppp['NONPROFIT'] = df_ppp['NONPROFIT'].fillna(False)

    # Change Veteran to boolean
    df_ppp['VETERAN'] = df_ppp['VETERAN'].map({'Veteran': True, 'Non-Veteran': False, 'Unanswered':None})

    # Sentence case the string columns
    df_ppp['BORROWER_ADDRESS'] = df_ppp['BORROWER_ADDRESS'].str.title()
    df_ppp['BORROWER_CITY'] = df_ppp['BORROWER_CITY'].str.title()
    df_ppp['ORIGINATING_LENDER_CITY'] = df_ppp['ORIGINATING_LENDER_CITY'].str.title()
    df_ppp['SERVICING_LENDER_CITY'] = df_ppp['SERVICING_LENDER_CITY'].str.title()
    df_ppp['PROJECT_COUNTY_NAME'] = df_ppp['PROJECT_COUNTY_NAME'].str.title()
    
    #df_ppp['LOAN_NUMBER'] = df_ppp['LOAN_NUMBER'].astype(int)
    df_ppp['DATE_APPROVED_ID'] = df_ppp['DATE_APPROVED_ID'].astype(int)
    df_ppp['SBA_OFFICE_CODE'] = df_ppp['SBA_OFFICE_CODE'].astype(int)
    df_ppp['PROCESSING_METHOD'] = df_ppp['PROCESSING_METHOD'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BORROWER_NAME'] = df_ppp['BORROWER_NAME'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BORROWER_ADDRESS'] = df_ppp['BORROWER_ADDRESS'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BORROWER_CITY'] = df_ppp['BORROWER_CITY'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BORROWER_STATE'] = df_ppp['BORROWER_STATE'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BORROWER_ZIP'] = df_ppp['BORROWER_ZIP'].astype(pd.StringDtype("pyarrow"))
    df_ppp['LOAN_STATUS_DATE_ID'] = df_ppp['LOAN_STATUS_DATE_ID'].astype(int)
    df_ppp['LOAN_STATUS'] = df_ppp['LOAN_STATUS'].astype(pd.StringDtype("pyarrow"))
    df_ppp['TERM_MONTH'] = df_ppp['TERM_MONTH'].astype(int)
    df_ppp['SBA_GUARANTY_PERCENTAGE'] = df_ppp['SBA_GUARANTY_PERCENTAGE'].astype(float)
    df_ppp['INITIAL_APPROVAL_AMOUNT'] = df_ppp['INITIAL_APPROVAL_AMOUNT'].astype(float)
    df_ppp['CURRENT_APPROVAL_AMOUNT'] = df_ppp['CURRENT_APPROVAL_AMOUNT'].astype(float)
    df_ppp['UNDISBURSED_AMOUNT'] = df_ppp['UNDISBURSED_AMOUNT'].astype(float)
    df_ppp['FRANCHISE_NAME'] = df_ppp['FRANCHISE_NAME'].astype(pd.StringDtype("pyarrow"))
    df_ppp['SERVICING_LENDER_LOCATION_ID'] = df_ppp['SERVICING_LENDER_LOCATION_ID'].astype(int)
    df_ppp['SERVICING_LENDER_NAME'] = df_ppp['SERVICING_LENDER_NAME'].astype(pd.StringDtype("pyarrow"))
    df_ppp['SERVICING_LENDER_ADDRESS'] = df_ppp['SERVICING_LENDER_ADDRESS'].astype(pd.StringDtype("pyarrow"))
    df_ppp['SERVICING_LENDER_CITY'] = df_ppp['SERVICING_LENDER_CITY'].astype(pd.StringDtype("pyarrow"))
    df_ppp['SERVICING_LENDER_STATE'] = df_ppp['SERVICING_LENDER_STATE'].astype(pd.StringDtype("pyarrow"))
    df_ppp['SERVICING_LENDER_ZIP'] = df_ppp['SERVICING_LENDER_ZIP'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BUSINESS_AGE_DESCRIPTION'] = df_ppp['BUSINESS_AGE_DESCRIPTION'].astype(pd.StringDtype("pyarrow"))
    df_ppp['PROJECT_STATE'] = df_ppp['PROJECT_STATE'].astype(pd.StringDtype("pyarrow"))
    df_ppp['PROJECT_COUNTY_NAME'] = df_ppp['PROJECT_COUNTY_NAME'].astype(pd.StringDtype("pyarrow"))
    df_ppp['RACE'] = df_ppp['RACE'].astype(pd.StringDtype("pyarrow"))
    df_ppp['ETHNICITY'] = df_ppp['ETHNICITY'].astype(pd.StringDtype("pyarrow"))
    df_ppp['GENDER'] = df_ppp['GENDER'].astype(pd.StringDtype("pyarrow"))
    df_ppp['BUSINESS_TYPE'] = df_ppp['BUSINESS_TYPE'].astype(pd.StringDtype("pyarrow"))
    df_ppp['ORIGINATING_LENDER_LOCATION_ID'] = df_ppp['ORIGINATING_LENDER_LOCATION_ID'].astype(int)
    df_ppp['ORIGINATING_LENDER'] = df_ppp['ORIGINATING_LENDER'].astype(pd.StringDtype("pyarrow"))
    df_ppp['ORIGINATING_LENDER_CITY'] = df_ppp['ORIGINATING_LENDER_CITY'].astype(pd.StringDtype("pyarrow"))
    df_ppp['ORIGINATING_LENDER_STATE'] = df_ppp['ORIGINATING_LENDER_STATE'].astype(pd.StringDtype("pyarrow"))
    df_ppp['VETERAN'] = df_ppp['VETERAN'].astype(bool)
    df_ppp['NONPROFIT'] = df_ppp['NONPROFIT'].astype(bool)
    df_ppp['FORGIVENESS_AMOUNT'] = df_ppp['FORGIVENESS_AMOUNT'].astype(float)
    df_ppp['FORGIVENESS_DATE_ID'] = df_ppp['FORGIVENESS_DATE_ID'].astype(int)
    df_ppp['JOBS_REPORTED'] = df_ppp['JOBS_REPORTED'].astype(int)
    df_ppp['NAICS_CODE'] = df_ppp['NAICS_CODE'].astype(int)

    # Create a FACTS_PPP_ID 
    df_ppp['FACTS_PPP_ID'] = range(1, len(df_ppp) + 1)

    return df_ppp

# Build the Dimensions and Facts Tables

In [12]:
clean_ppp_data = reformat_ppp_loan_data()
clean_naics_data = reformat_naics_data()
clean_gdp_data = reformat_gdp_data()

Downloaded public_150k_plus_230930.csv successfully
Reading public_150k_plus_230930.csv


In [13]:
# Create the dimensions
dim_naics = reformat_naics_data() # Completed
dim_naics.head()

,NAICS_CODE,NAICS_TITLE,DESCRIPTION
0,11,"Agriculture, Forestry, Fishing and HuntingT","The Sector as a Whole The Agriculture, Forest..."
1,111,Crop ProductionT,Industries in the Crop Production subsector gr...
2,1111,Oilseed and Grain FarmingT,This industry group comprises establishments p...
3,11111,Soybean FarmingT,See industry description for 111110.
4,111110,Soybean Farming,This industry comprises establishments primari...


In [14]:
dim_sba_office = clean_ppp_data[['SBA_OFFICE_CODE']].drop_duplicates()
dim_sba_office = dim_sba_office.reset_index(drop=True)
dim_sba_office.head()

,SBA_OFFICE_CODE
0,1084
1,459
2,470
3,405
4,669


In [15]:
"""
GEOFIPS and GEO_NAME are from the GDP data
PROJECT_COUNTY_NAME and PROJECT_STATE are a needed reference for the PPP data

If the GEO_FIPS = 0, then it is the United States
Project State will be "All States", and Project County will be "All Counties"

If the GEO_FIPS ends with "000", then it is a state. 
So PROJECT_STATE will be the GEO_NAME, and the PROJECT_COUNTY_NAME will be "All Counties"

for anything else, we can split the GEO_NAME by ", " and get the first part as the PROJECT_COUNTY_NAME and the second part as the PROJECT_STATE
"""
dim_geography = clean_gdp_data[['GEOFIPS', 'GEO_NAME', 'Region']].drop_duplicates()
dim_geography = dim_geography.reset_index(drop=True)

dim_geography['PROJECT_STATE'] = dim_geography['GEO_NAME'].str.split(',').str[1].str.strip()
dim_geography['PROJECT_COUNTY_NAME'] = dim_geography['GEO_NAME'].str.split(',').str[0].str.strip()

# Temporarily set GEO_FIPS to string
dim_geography['GEOFIPS'] = dim_geography['GEOFIPS'].astype(str)

# Set the PROJECT_STATE and PROJECT_COUNTY_NAME for the United States
dim_geography.loc[dim_geography['GEOFIPS'] == '0', 'PROJECT_STATE'] = 'All States'
dim_geography.loc[dim_geography['GEOFIPS'] == '0', 'PROJECT_COUNTY_NAME'] = 'All Counties'

# Set the PROJECT_STATE and PROJECT_COUNTY_NAME for the States
dim_geography.loc[dim_geography['GEOFIPS'].str.endswith('000'), 'PROJECT_STATE'] = dim_geography['GEO_NAME']
dim_geography.loc[dim_geography['GEOFIPS'].str.endswith('000'), 'PROJECT_COUNTY_NAME'] = 'All Counties'

# Reset the GEO_FIPS to int
dim_geography['GEOFIPS'] = dim_geography['GEOFIPS'].astype(int)

# In the clean_ppp_data, create the GEONAME column using the PROJECT_STATE and PROJECT_COUNTY_NAME
clean_ppp_data['GEO_NAME'] = clean_ppp_data['PROJECT_COUNTY_NAME'] + ', ' + clean_ppp_data['PROJECT_STATE']

#Show me the PROJECT_STATE and PROJECT_COUNTY_NAME, GEO_NAME head in the clean_ppp_data
clean_ppp_data[['PROJECT_STATE', 'PROJECT_COUNTY_NAME', 'GEO_NAME']].head()


# Merge the clean_ppp_data with the dim_geography to get the GEO_FIPS
clean_ppp_data = clean_ppp_data.merge(dim_geography[['GEO_NAME', 'GEOFIPS']], on='GEO_NAME', how='left', suffixes=('', '_dim_geography'))
# Make GEO_FIPS as int

dim_geography.tail()

Description,GEOFIPS,GEO_NAME,Region,PROJECT_STATE,PROJECT_COUNTY_NAME
3167,94000,Plains,4,Plains,All Counties
3168,95000,Southeast,5,Southeast,All Counties
3169,96000,Southwest,6,Southwest,All Counties
3170,97000,Rocky Mountain,7,Rocky Mountain,All Counties
3171,98000,Far West,8,Far West,All Counties


In [16]:
dim_originating_lender = clean_ppp_data[['ORIGINATING_LENDER_LOCATION_ID', 'ORIGINATING_LENDER', 'ORIGINATING_LENDER_CITY', 'ORIGINATING_LENDER_STATE']].drop_duplicates()
dim_originating_lender["ORIGINATING_LENDER_ID"] = range(1, len(dim_originating_lender) + 1)
# Change column order
dim_originating_lender = dim_originating_lender[['ORIGINATING_LENDER_ID', 'ORIGINATING_LENDER_LOCATION_ID', 'ORIGINATING_LENDER', 'ORIGINATING_LENDER_CITY', 'ORIGINATING_LENDER_STATE']]
dim_originating_lender = dim_originating_lender.reset_index(drop=True)

clean_ppp_data = clean_ppp_data.merge(dim_originating_lender[['ORIGINATING_LENDER_LOCATION_ID', 'ORIGINATING_LENDER_ID']], on='ORIGINATING_LENDER_LOCATION_ID', how='left', suffixes=('', '_dim_originating_lender'))

dim_originating_lender.head()

,ORIGINATING_LENDER_ID,ORIGINATING_LENDER_LOCATION_ID,ORIGINATING_LENDER,ORIGINATING_LENDER_CITY,ORIGINATING_LENDER_STATE
0,1,116975,Northrim Bank,Anchorage,AK
1,2,89628,"National Cooperative Bank, National Association",Hillsboro,OH
2,3,3386,First National Bank Alaska,Anchorage,AK
3,4,119918,East West Bank,Pasadena,CA
4,5,194499,CoBank ACB,Greenwood Village,CO


In [17]:
dim_borrower = clean_ppp_data[['BORROWER_NAME', 'BORROWER_ADDRESS', 'BORROWER_CITY', 'BORROWER_STATE', 'BORROWER_ZIP', 'RACE', 'ETHNICITY', 'GENDER', 'VETERAN', 'FRANCHISE_NAME', 'NONPROFIT', 'JOBS_REPORTED']].drop_duplicates()
dim_borrower["BORROWER_ID"] = range(1, len(dim_borrower) + 1)
# Change the column order
dim_borrower = dim_borrower.reset_index(drop=True)

clean_ppp_data = clean_ppp_data.merge(dim_borrower[['BORROWER_NAME', 'BORROWER_ADDRESS', 'BORROWER_CITY', 'BORROWER_STATE', 'BORROWER_ZIP', 'BORROWER_ID']], on=['BORROWER_NAME', 'BORROWER_ADDRESS', 'BORROWER_CITY', 'BORROWER_STATE', 'BORROWER_ZIP'], how='left', suffixes=('', '_dim_borrower'))
dim_borrower.head()

,BORROWER_NAME,BORROWER_ADDRESS,BORROWER_CITY,BORROWER_STATE,BORROWER_ZIP,RACE,ETHNICITY,GENDER,VETERAN,FRANCHISE_NAME,NONPROFIT,JOBS_REPORTED,BORROWER_ID
0,"KAKIVIK ASSET MANAGEMENT, LLC",5015 Business Park Blvd,Anchorage,AK,99503-7146,Unanswered,Unknown/NotStated,Unanswered,False,<NA>,False,385,1
1,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,Barrow,AK,99723,Unanswered,Unknown/NotStated,Unanswered,False,<NA>,True,295,2
2,HOPE COMMUNITY RESOURCES INC.,540 W Intl Airport Rd,Anchorage,AK,99518-1105,Unanswered,Unknown/NotStated,Unanswered,False,<NA>,True,500,3
3,SOUTH PENINSULA HOSPITAL INC,4300 Bartlett Street,Homer,AK,99603,Unanswered,Unknown/NotStated,Unanswered,False,<NA>,False,439,4
4,"COPPER RIVER SEAFOODS, INC.",1118 5Th Ave,Anchorage,AK,99501-2759,Unanswered,Unknown/NotStated,Male Owned,False,<NA>,False,303,5


In [18]:
dim_servicing_lender = clean_ppp_data[['SERVICING_LENDER_LOCATION_ID', 'SERVICING_LENDER_NAME', 'SERVICING_LENDER_ADDRESS', 'SERVICING_LENDER_CITY', 'SERVICING_LENDER_STATE', 'SERVICING_LENDER_ZIP']].drop_duplicates()
dim_servicing_lender["SERVICING_LENDER_ID"] = range(1, len(dim_servicing_lender) + 1)
# Change the column order
dim_servicing_lender = dim_servicing_lender.reset_index(drop=True)
dim_servicing_lender = dim_servicing_lender[['SERVICING_LENDER_ID', 'SERVICING_LENDER_LOCATION_ID', 'SERVICING_LENDER_NAME', 'SERVICING_LENDER_ADDRESS', 'SERVICING_LENDER_CITY', 'SERVICING_LENDER_STATE', 'SERVICING_LENDER_ZIP']]
clean_ppp_data = clean_ppp_data.merge(dim_servicing_lender[['SERVICING_LENDER_LOCATION_ID', 'SERVICING_LENDER_ID']], on='SERVICING_LENDER_LOCATION_ID', how='left', suffixes=('', '_dim_servicing_lender'))
dim_servicing_lender.head()

,SERVICING_LENDER_ID,SERVICING_LENDER_LOCATION_ID,SERVICING_LENDER_NAME,SERVICING_LENDER_ADDRESS,SERVICING_LENDER_CITY,SERVICING_LENDER_STATE,SERVICING_LENDER_ZIP
0,1,116975,Northrim Bank,3111 'C' St,Anchorage,AK,99503
1,2,89628,"National Cooperative Bank, National Association",139 S High St,Hillsboro,OH,45133-1442
2,3,3386,First National Bank Alaska,101 W 36th Ave,Anchorage,AK,99503-5904
3,4,119918,East West Bank,"135 N Los Robles Ave, 7th Fl",Pasadena,CA,91101-4525
4,5,194499,CoBank ACB,6340 S Fiddlers Green Cir #1908,Greenwood Village,CO,80111


Use Factorize to create tables:

In [19]:
dim_loan_status = clean_ppp_data[['LOAN_STATUS']].drop_duplicates()
dim_loan_status["LOAN_STATUS_ID"] = range(1, len(dim_loan_status) + 1)
dim_loan_status = dim_loan_status[['LOAN_STATUS_ID', 'LOAN_STATUS']]
dim_loan_status = dim_loan_status.reset_index(drop=True)
clean_ppp_data = clean_ppp_data.merge(dim_loan_status[['LOAN_STATUS', 'LOAN_STATUS_ID']], on='LOAN_STATUS', how='left', suffixes=('', '_dim_loan_status'))
dim_loan_status.head()


,LOAN_STATUS_ID,LOAN_STATUS
0,1,Paid in Full
1,2,Charged Off


In [20]:
dim_business_type = clean_ppp_data[['BUSINESS_TYPE']].drop_duplicates()
dim_business_type["BUSINESS_TYPE_ID"] = range(1, len(dim_business_type) + 1)
dim_business_type = dim_business_type[['BUSINESS_TYPE_ID', 'BUSINESS_TYPE']]
dim_business_type = dim_business_type.reset_index(drop=True)

clean_ppp_data = clean_ppp_data.merge(dim_business_type[['BUSINESS_TYPE', 'BUSINESS_TYPE_ID']], on='BUSINESS_TYPE', how='left', suffixes=('', '_dim_business_type'))
dim_business_type.head()

,BUSINESS_TYPE_ID,BUSINESS_TYPE
0,1,Limited Liability Company(LLC)
1,2,Non-Profit Organization
2,3,501(c)3 – Non Profit
3,4,Corporation
4,5,Cooperative


In [21]:
dim_processing_method = clean_ppp_data[['PROCESSING_METHOD']].drop_duplicates()
dim_processing_method["PROCESSING_METHOD_ID"] = range(1, len(dim_processing_method) + 1)
dim_processing_method = dim_processing_method[['PROCESSING_METHOD_ID', 'PROCESSING_METHOD']]
dim_processing_method = dim_processing_method.reset_index(drop=True)

clean_ppp_data = clean_ppp_data.merge(dim_processing_method[['PROCESSING_METHOD', 'PROCESSING_METHOD_ID']], on='PROCESSING_METHOD', how='left', suffixes=('', '_dim_processing_method'))

dim_processing_method

,PROCESSING_METHOD_ID,PROCESSING_METHOD
0,1,PPP
1,2,PPS


In [22]:
dim_term = clean_ppp_data[['TERM_MONTH']].drop_duplicates()
dim_term = dim_term.sort_values(by='TERM_MONTH')
dim_term["TERM_ID"] = range(1, len(dim_term) + 1)
dim_term = dim_term[['TERM_ID', 'TERM_MONTH']]
dim_term = dim_term.reset_index(drop=True)

clean_ppp_data = clean_ppp_data.merge(dim_term[['TERM_MONTH', 'TERM_ID']], on='TERM_MONTH', how='left', suffixes=('', '_dim_term'))

dim_term.head()

,TERM_ID,TERM_MONTH
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4


In [23]:
dim_business_age = clean_ppp_data[['BUSINESS_AGE_DESCRIPTION']].drop_duplicates()
dim_business_age["BUSINESS_AGE_ID"] = range(1, len(dim_business_age) + 1)
dim_business_age = dim_business_age[['BUSINESS_AGE_ID', 'BUSINESS_AGE_DESCRIPTION']]
dim_business_age = dim_business_age.reset_index(drop=True)

clean_ppp_data = clean_ppp_data.merge(dim_business_age[['BUSINESS_AGE_DESCRIPTION', 'BUSINESS_AGE_ID']], on='BUSINESS_AGE_DESCRIPTION', how='left', suffixes=('', '_dim_business_age'))

In [24]:
"""
PPP Fact Table
Columns:
FACTS_PPP_ID (integer, primary key) - from the PPP data
LOAN_NUMBER (integer) - PPP data
NAICS_CODE (integer, foreign key) - PPP data
GEOFIPS (integer, foreign key) - Using PROJECT_STATE and PROJECT_COUNTY_NAME PPP data, find the GEOFIPS from the Geography Dimension
DATE_APPROVED_ID (integer, foreign key) - PPP data
LOAN_STATUS_DATE_ID (integer, foreign key) - PPP data
FORGIVENESS_DATE_ID (integer, foreign key) - PPP data
BORROWER_ID (integer, foreign key) - Find the BORROWER_ID from the Borrower Dimension
ORIGINATING_LENDER_ID (integer, foreign key) - Find the ORIGINATING_LENDER_ID from the Originating Lender Dimension
SERVICING_LENDER_ID (integer, foreign key) - Find the SERVICING_LENDER_ID from the Servicing Lender Dimension
TERM_ID (integer, foreign key) - Find the TERM_ID from the Term Dimension
LOAN_STATUS_ID (integer, foreign key) - Find the LOAN_STATUS_ID from the Loan Status Dimension
PROCESSING_METHOD_ID (integer, foreign key) - Find the PROCESSING_METHOD_ID from the Processing Method Dimension
SBA_OFFICE_CODE (integer, foreign key) - PPP data
BUSINESS_AGE_ID (integer, foreign key) - Find the BUSINESS_AGE_ID from the Business Age Dimension
BUSINESS_TYPE_ID (integer, foreign key) - Find the BUSINESS_TYPE_ID from the Business Type Dimension
SBA_GUARANTY_PERCENTAGE (number) - PPP data
INITIAL_APPROVAL_AMOUNT (number) - PPP data
CURRENT_APPROVAL_AMOUNT (number) - PPP data
UNDISBURSED_AMOUNT (number) - PPP data
FORGIVENESS_AMOUNT (number) - PPP data
"""
fact_ppp = clean_ppp_data[['FACTS_PPP_ID', 'LOAN_NUMBER', 'NAICS_CODE', 'GEOFIPS', 'DATE_APPROVED_ID', 'LOAN_STATUS_DATE_ID', 'FORGIVENESS_DATE_ID', 'BORROWER_ID', 'ORIGINATING_LENDER_ID', 'SERVICING_LENDER_ID', 'TERM_ID', 'LOAN_STATUS_ID', 'PROCESSING_METHOD_ID', 'SBA_OFFICE_CODE', 'BUSINESS_AGE_ID', 'BUSINESS_TYPE_ID', 'SBA_GUARANTY_PERCENTAGE', 'INITIAL_APPROVAL_AMOUNT', 'CURRENT_APPROVAL_AMOUNT', 'UNDISBURSED_AMOUNT', 'FORGIVENESS_AMOUNT']]
fact_ppp = fact_ppp.reset_index(drop=True)
fact_ppp.tail()

,FACTS_PPP_ID,LOAN_NUMBER,NAICS_CODE,GEOFIPS,DATE_APPROVED_ID,LOAN_STATUS_DATE_ID,FORGIVENESS_DATE_ID,BORROWER_ID,ORIGINATING_LENDER_ID,SERVICING_LENDER_ID,...,LOAN_STATUS_ID,PROCESSING_METHOD_ID,SBA_OFFICE_CODE,BUSINESS_AGE_ID,BUSINESS_TYPE_ID,SBA_GUARANTY_PERCENTAGE,INITIAL_APPROVAL_AMOUNT,CURRENT_APPROVAL_AMOUNT,UNDISBURSED_AMOUNT,FORGIVENESS_AMOUNT
917733,878865,4395967002,621210,56025.0,2020040300,2021011300,2020120900,873200,1431,1322,...,1,1,897,1,4,100.0,150000.0,150000.0,0.0,151037.50
917734,878866,6985647108,624410,56037.0,2020041400,2020120800,2020110300,873201,1538,1417,...,1,1,897,1,12,100.0,150000.0,150000.0,0.0,150789.04
917735,878867,7996438405,238210,56005.0,2021021200,2021091500,2021081700,873202,666,621,...,1,2,897,1,6,100.0,150000.0,150000.0,0.0,150743.84
917736,878868,9054647103,621610,56033.0,2020041500,2022030800,2022022300,873203,844,791,...,1,1,897,2,4,100.0,150000.0,150000.0,0.0,152820.83
917737,878869,9184687004,722511,56039.0,2020040900,2021102200,2021092200,873204,33,33,...,1,1,897,1,6,100.0,150000.0,150000.0,0.0,152162.50


In [25]:
# Create the Fact Table
facts_gdp = clean_gdp_data[['FACTS_GDP_ID', 'YEAR_ID', 'REAL_GDP', 'CHAIN_TYPE_INDEX_GDP', 'CURRENT_DOLLAR_GDP', 'GEOFIPS']]
facts_gdp.head(20)

Description,FACTS_GDP_ID,YEAR_ID,REAL_GDP,CHAIN_TYPE_INDEX_GDP,CURRENT_DOLLAR_GDP,GEOFIPS
0,1,2017010100,1.961210e+10,100.000,1.961210e+10,0
1,2,2018010100,2.019390e+10,102.967,2.065652e+10,0
2,3,2019010100,2.069209e+10,105.507,2.152140e+10,0
3,4,2020010100,2.023407e+10,103.171,2.132295e+10,0
4,5,2021010100,2.140769e+10,109.156,2.359403e+10,0
5,6,2022010100,2.182204e+10,111.268,2.574411e+10,0
6,7,2017010100,2.166155e+08,100.000,2.166155e+08,1000
7,8,2018010100,2.208088e+08,101.936,2.262638e+08,1000
8,9,2019010100,2.249446e+08,103.845,2.345264e+08,1000
9,10,2020010100,2.220814e+08,102.523,2.351183e+08,1000


### Date Dimension
Start date: 2017-01-01 00:00:00 

2017 is the minimum year in the GDP data

End date: 2023-10-1 00:00:00 

October 2023 is the maximum date in the PPP data

In [26]:
def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

start_date = pd.to_datetime('2017-01-01 00:00:00')
end_date = pd.to_datetime('2023-10-01 00:00:00')
# Create a DataFrame for the date dimension
date_dimension = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='H')})

# Extract attributes
date_dimension['YEAR_NUMBER'] = date_dimension['date'].dt.year
date_dimension['QUARTER_NUMBER'] = date_dimension['date'].dt.quarter #QUARTER_NUMBER
date_dimension['MONTH_NUMBER'] = date_dimension['date'].dt.month
date_dimension['MONTH_NAME'] = date_dimension['date'].dt.strftime('%B')
date_dimension['DAY_NUMBER'] = date_dimension['date'].dt.day #DAY_NUMBER
date_dimension['DAY_NAME'] = date_dimension['date'].dt.strftime('%A') #DAY_NAME
date_dimension['HOUR_NUMBER'] = date_dimension['date'].dt.hour #HOUR_NUMBER
date_dimension['DATE_ISO_FORMAT'] = date_dimension['date'].apply(lambda x: x.isoformat())
date_dimension['DATE_ID'] = date_dimension['date'].dt.strftime('%Y%m%d%H')

# Add week of the month and week of the year
date_dimension['WEEK_OF_MONTH'] = date_dimension['date'].apply(week_of_month) #WEEK_OF_MONTH
date_dimension['WEEK_OF_YEAR'] = date_dimension['date'].dt.strftime('%U') #WEEK_OF_YEAR

new_order = ['DATE_ID', 'DATE_ISO_FORMAT','YEAR_NUMBER','QUARTER_NUMBER','MONTH_NUMBER','DAY_NUMBER','HOUR_NUMBER','MONTH_NAME','DAY_NAME','WEEK_OF_YEAR','WEEK_OF_MONTH']
date_dimension = date_dimension[new_order]

date_dimension.head(10)

,DATE_ID,DATE_ISO_FORMAT,YEAR_NUMBER,QUARTER_NUMBER,MONTH_NUMBER,DAY_NUMBER,HOUR_NUMBER,MONTH_NAME,DAY_NAME,WEEK_OF_YEAR,WEEK_OF_MONTH
0,2017010100,2017-01-01T00:00:00,2017,1,1,1,0,January,Sunday,01,1
1,2017010101,2017-01-01T01:00:00,2017,1,1,1,1,January,Sunday,01,1
2,2017010102,2017-01-01T02:00:00,2017,1,1,1,2,January,Sunday,01,1
3,2017010103,2017-01-01T03:00:00,2017,1,1,1,3,January,Sunday,01,1
4,2017010104,2017-01-01T04:00:00,2017,1,1,1,4,January,Sunday,01,1
5,2017010105,2017-01-01T05:00:00,2017,1,1,1,5,January,Sunday,01,1
6,2017010106,2017-01-01T06:00:00,2017,1,1,1,6,January,Sunday,01,1
7,2017010107,2017-01-01T07:00:00,2017,1,1,1,7,January,Sunday,01,1
8,2017010108,2017-01-01T08:00:00,2017,1,1,1,8,January,Sunday,01,1
9,2017010109,2017-01-01T09:00:00,2017,1,1,1,9,January,Sunday,01,1


# Transform the Data 

# Save the data frames as CSV  

# Load Data into the Data Warehouse

# Main

In [27]:
if __name__ == "__main__":
    pass